In [7]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [35]:
learning_rate = 0.0001
height = 28   #image height
weight = 28   #image weight
channels = 1  #black and white
n_inputs = height * weight    #input nodes

conv1_fmaps = 32  #the number of maps
conv1_ksize = 3   #kernel size 3*3
conv1_stride = 1 # means weight and height both are 1 stride forward each piece.remember the strange picture, 1 filter 1 map 
                    # each map is a strange picture(physical way of human vision)    1 * 1 stride
conv1_pad = "SAME" #"SAME" padding "valid" no padding

conv2_fmaps = 64 #the number of maps
conv2_ksize = 3 
conv2_stride = 2
conv2_pad = "SAME"

maxpool_ksize = [1, 2, 2, 1]
maxpool_stride = [1, 2, 2, 1]
maxpool_pad = "VALID"
conv2_dropout_rate = 0.25
fc1_dropout = 0.5
pool3_fmap = conv2_fmaps

output_size = 10

tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, weight, channels])
    Y = tf.placeholder(tf.float32, shape=[None, output_size], name="Y")
    training = tf.placeholder_with_default(False, shape=[], name='training')
with tf.name_scope("conv"):
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize, strides=conv1_stride, 
                             padding=conv1_pad, activation=tf.nn.relu, name="conv1")
    pool1 = tf.nn.max_pool(conv1, ksize=maxpool_ksize, strides=maxpool_stride, padding=maxpool_pad)
    conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize, strides=conv2_stride, 
                             padding=conv2_pad, activation=tf.nn.relu, name="conv2")
    
with tf.name_scope("max_pool"):
    pool3 = tf.nn.max_pool(conv2, ksize=maxpool_ksize, strides=maxpool_stride, padding=maxpool_pad)
    shape = pool3.shape
    #shape = conv2.shape
    after_height = shape[1]
    after_weight = shape[2]
    after_channel = shape[3]
    flatten_fc_input = tf.reshape(pool3, shape=[-1, after_height*after_weight*after_channel])
    pool3_flat_drop = tf.layers.dropout(flatten_fc_input, conv2_dropout_rate, training=training)
    fc_output_size = 128 #(after_height * after_weight * after_channel) // 2
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flatten_fc_input, fc_output_size, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout, training=training)
    logits = tf.layers.dense(fc1_drop, output_size, name="logits")

with tf.name_scope("prediction"):
    y_proba = tf.nn.softmax(logits, name="probability")  #data normalization's prediction

with tf.name_scope("train"):
    #Y_change = tf.reshape(Y,(-1,1))
    output = tf.argmax(y_proba, axis=1)
    targets = tf.argmax(Y, axis=1)
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)   # normalization and loss function
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, targets, 1)  #top 1 evaluation
    #correct_prediction = tf.equal(output, targets)
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
from  sklearn.preprocessing import OneHotEncoder
def onehot(X):
    encoder = OneHotEncoder()
    Y = encoder.fit(X.reshape(-1,1))
    #Y = Y*0.98
    #Y = Y.toarray() + 0.01
    return Y
#X_batch, Y_batch = mnist.train.next_batch(1)
#a = onehot(Y_batch)
#a
encoder = onehot(mnist.train.labels)
def datapreprocessing(X, encoder):
    Y = encoder.transform(X)
    Y = Y * 0.98
    Y = Y.toarray() + 0.01
    return Y

In [36]:
epochs = 100
batch_size = 100
best_value = 0
max_tolerate = 10000#20np.infty
without_process = 0
#X_train, Y_train_orignal = mnist.train.images, mnist.train.labels
#Y_train = datapreprocessing(Y_train_orignal.reshape(-1,1), encoder)
rounds = mnist.train.num_examples // batch_size
with tf.Session() as sess:
    init.run()
    for epoch in range(epochs): #epochs
        for iteration in range(rounds): #mnist.train.num_examples // batch_size
            X_batch, Y_orignal = mnist.train.next_batch(batch_size)
            Y_batch = datapreprocessing(Y_orignal.reshape(-1,1), encoder)
            #Y_batch = Y_batch.reshape(-1,1)
            #la = sess.run(xentropy, feed_dict={X:X_batch, Y:Y_batch})
            #print(la)
            #a, b, c, d = sess.run([y_proba, targets, training_op, correct], feed_dict={X:X_batch, Y:Y_batch})
            #print(b, "\n\n\n")
            #print(Y_orignal)
            a, acc_train = sess.run([training_op, accuracy], feed_dict={X:X_batch, Y:Y_batch})
            Y_validation = datapreprocessing(mnist.validation.labels.reshape(-1,1), encoder)
            #if iteration % 100 == 0:
            acc_validation = sess.run(accuracy, feed_dict={X:mnist.validation.images, Y:Y_validation})
                    #print(d)
            if acc_validation > best_value:
                best_value = acc_validation
                save_path = saver.save(sess, "./cnn_mnist_no_pooling.ckpt")
                without_process = 0
            else:
                without_process += 1
            
            if iteration % 10 == 0:
                print(epoch,"/",epochs, iteration,"/",rounds, "Train accuracy:", acc_train, "Validation accuracy:", acc_validation, "Best value:", best_value)
                
        if without_process > max_tolerate:
            print("early stop!")
            break
        #acc_train = sess.run(accuracy, feed_dict={X:X_batch, Y:Y_batch})

0 Train accuracy: 0.08 Validation accuracy: 0.1036 Best value: 0.1036
0 Train accuracy: 0.2 Validation accuracy: 0.1894 Best value: 0.1894
0 Train accuracy: 0.2 Validation accuracy: 0.26 Best value: 0.26
0 Train accuracy: 0.37 Validation accuracy: 0.3578 Best value: 0.3578
0 Train accuracy: 0.54 Validation accuracy: 0.456 Best value: 0.456
0 Train accuracy: 0.49 Validation accuracy: 0.4994 Best value: 0.4994
0 Train accuracy: 0.57 Validation accuracy: 0.564 Best value: 0.564
0 Train accuracy: 0.57 Validation accuracy: 0.5614 Best value: 0.566
0 Train accuracy: 0.48 Validation accuracy: 0.5916 Best value: 0.5916
0 Train accuracy: 0.63 Validation accuracy: 0.6398 Best value: 0.6398
0 Train accuracy: 0.6 Validation accuracy: 0.6546 Best value: 0.66
0 Train accuracy: 0.64 Validation accuracy: 0.6578 Best value: 0.66
0 Train accuracy: 0.65 Validation accuracy: 0.6712 Best value: 0.6726
0 Train accuracy: 0.67 Validation accuracy: 0.6678 Best value: 0.6726
0 Train accuracy: 0.67 Validation ac

KeyboardInterrupt: 

In [13]:
Y_batch.shape

(100, 10)

In [30]:
mnist.train.images[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  